::AIM::
In here we will take in and work out how to make sense of the data that is stored from the GameLogic script.
The aim of this is to see if the data file contains all we need.
::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

1# We import csv

2# We specify the filepath

3# We open the file and print out the data of each row.

In [28]:
import csv
file = "../results/data.csv"

with open(file,"r") as data:
    reader = csv.reader(data)
    for row in reader:
        print(row)

['1', 'shape', 'shape', '1']
['1', 'shape', 'shape', '2']
['1', 'shape', 'shape', '3']
['1', 'shape', 'shape', '4']
['1', 'shape', 'shape', '5']
['0', 'shapenumber', 'color', '0']
['1', 'colorshape', 'color', '1']
['1', 'color', 'color', '2']
['1', 'colornumber', 'color', '3']
['0', 'NONE', 'color', '0']
['1', 'color', 'color', '1']
['1', 'colorshape', 'color', '2']
['1', 'color', 'color', '3']
['1', 'color', 'color', '4']
['1', 'colorshape', 'color', '5']
['1', 'colorshape', 'shape', '1']
['1', 'shapenumber', 'shape', '2']
['1', 'shapenumber', 'shape', '3']
['1', 'shape', 'shape', '4']
['1', 'shape', 'shape', '5']
['0', 'shape', 'number', '0']
['0', 'shape', 'number', '0']
['0', 'shape', 'number', '0']
['0', 'shape', 'number', '0']
['1', 'number', 'number', '1']
['1', 'number', 'number', '2']
['0', 'NONE', 'number', '0']
['0', 'NONE', 'number', '0']
['0', 'shape', 'number', '0']
['0', 'NONE', 'number', '0']
['1', 'colornumber', 'number', '1']
['1', 'number', 'number', '2']
['1', 'colo

Now instead of printing it we unpack the values into tuples and save them to a list.

In [29]:
results = []
with open(file,"r") as data:
    reader = csv.reader(data)
    for row in reader:
        results.append(tuple(row))

print(results)
            

[('1', 'shape', 'shape', '1'), ('1', 'shape', 'shape', '2'), ('1', 'shape', 'shape', '3'), ('1', 'shape', 'shape', '4'), ('1', 'shape', 'shape', '5'), ('0', 'shapenumber', 'color', '0'), ('1', 'colorshape', 'color', '1'), ('1', 'color', 'color', '2'), ('1', 'colornumber', 'color', '3'), ('0', 'NONE', 'color', '0'), ('1', 'color', 'color', '1'), ('1', 'colorshape', 'color', '2'), ('1', 'color', 'color', '3'), ('1', 'color', 'color', '4'), ('1', 'colorshape', 'color', '5'), ('1', 'colorshape', 'shape', '1'), ('1', 'shapenumber', 'shape', '2'), ('1', 'shapenumber', 'shape', '3'), ('1', 'shape', 'shape', '4'), ('1', 'shape', 'shape', '5'), ('0', 'shape', 'number', '0'), ('0', 'shape', 'number', '0'), ('0', 'shape', 'number', '0'), ('0', 'shape', 'number', '0'), ('1', 'number', 'number', '1'), ('1', 'number', 'number', '2'), ('0', 'NONE', 'number', '0'), ('0', 'NONE', 'number', '0'), ('0', 'shape', 'number', '0'), ('0', 'NONE', 'number', '0'), ('1', 'colornumber', 'number', '1'), ('1', 'num

Then lets start simple analysis.

We have to keep in mind the that the tuple indexing carries with it information about the values:
index: 0 : correct = 1, wrong = 0
index: 1 : the category on which the chosen stimulus card and hand card were matched on
index: 2 : the active rule
index: 3 : winning streak

In [30]:
def correct_procent(results):
    cumulative = 0
    for n, data_point in enumerate(results):
        cumulative += int(data_point[0])
    correct_procent = cumulative / (n + 1)
    print(correct_procent)
    
correct_procent(results)

0.71875


In [31]:
def completed_categories(results):
    completed = 0
    for data_point in results:
        if data_point[3] == "5":
            completed += 1
    return completed

print(completed_categories(results))
        

6


This next one is tricky. I used the logic that if a participant sorts with the same wrong rule consequtively, they get 1 point on preservative errors. When the situation is less clear, as when the card is matched on a wrong rule but there is also included another wrong rule: exmaple: shape and shapenumber. Then 0.5 is added to the preservation error. The logic behind it is that we dont know if the participant is testing out a new rule or if they are making a preservation error.

Note: how this is analysed is hidden behind a paywall. So this is in a sense only a dummy variable that follows approximately the same logic as the one used in the commercial / clinical testings. 

In the function is also a documentation printer: this way it gives an intuitive view into what happened during the experiment. This was made for debugging and checking if this actually is doing what i wanted.

One thing to note was that i included no matching category as a sorting rule. In a sense if a card is matched on no category that could also be seen as a sorting rule, and if participants try to sort according to this rule, even after negative feedback. It is treated as a preservation error.

In [32]:
def preservative_errors(results):
    
    preservative_error = 0
    error_streak = 0
    wrong_rule = None
    
    for data_point in results:
        if error_streak >= 1 and data_point[1] == wrong_rule:
            preservative_error += 1
            error_streak += 1
            print(f"used rule:{data_point[1]},previous_used_rule:{wrong_rule}, error + 1")
        
        elif error_streak >= 1 and data_point[1] in wrong_rule:
            preservative_error += 0.5
            error_streak += 1
            print(f"used rule:{data_point[1]},previous_used_rule:{wrong_rule}, error + 0.5")
            
        
        elif int(data_point[3]) == 0:
            error_streak += 1
            wrong_rule = data_point[1]
            print(f"sorted by wrong rule:{wrong_rule}")
            
        else:
            error_streak = 0
            wrong_rule = None
            print("correct")
            
    return preservative_error

print(preservative_errors(results))

correct
correct
correct
correct
correct
sorted by wrong rule:shapenumber
correct
correct
correct
sorted by wrong rule:NONE
correct
correct
correct
correct
correct
correct
correct
correct
correct
correct
sorted by wrong rule:shape
used rule:shape,previous_used_rule:shape, error + 1
used rule:shape,previous_used_rule:shape, error + 1
used rule:shape,previous_used_rule:shape, error + 1
correct
correct
sorted by wrong rule:NONE
used rule:NONE,previous_used_rule:NONE, error + 1
sorted by wrong rule:shape
sorted by wrong rule:NONE
correct
correct
correct
sorted by wrong rule:colorshape
correct
correct
correct
correct
correct
sorted by wrong rule:shapenumber
used rule:number,previous_used_rule:shapenumber, error + 0.5
correct
correct
sorted by wrong rule:NONE
correct
correct
correct
correct
correct
sorted by wrong rule:colornumber
used rule:color,previous_used_rule:colornumber, error + 0.5
correct
correct
correct
correct
correct
correct
correct
correct
sorted by wrong rule:colorshape
used rul

next we will make a function that measures how many errors participants are maintaining sorting according to a correct rule after they have gotten positive feedback on that rule.